# This notebook will be mainly used for the capstone project

In [3]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Scrapping the wikipedia page of Toronto

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url).read()
bs = BeautifulSoup(html)
table = bs.findAll('table')

In [6]:
rows = table[0].findAll('tr')

In [7]:
df = pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])

In [8]:
for r in rows:
    elements = r.findAll('td')
    toadd = []
    for e in elements:
        toadd.append(e.text)
    if len(toadd) == 3:
        df = df.append({'PostalCode': toadd[0][0:-1], 'Borough': toadd[1][0:-1], 'Neighborhood': toadd[2][0:-1]}, ignore_index=True)

In [9]:
df = df[df['Borough'] != 'Not assigned']
df = df.reset_index(drop = True)

In [10]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
df.shape

(103, 3)

# Get Latitude and the Longitude of Neighborhoods

In [31]:
# loop until you get the coordinates
lat = []
lon = []
for index, row in df.iterrows():
    print(index)
    location = geolocator.geocode('{}, {}, Toronto, Ontario'.format(row['Neighborhood'], row['Borough']))
    if location is None:
        location = geolocator.geocode('{}, Toronto, Ontario'.format(row['Borough']))
    lat.append(location.latitude)
    lon.append(location.longitude)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102


In [33]:
lat = np.asarray(lat, dtype = float)
print(lat)

[43.7587999  43.732658   43.6563221  43.7543263  43.6563221  43.6684992
 43.8091955  43.775347   43.699971   43.6563221  43.7087117  43.6435559
 43.773077   43.775347   43.69992    43.6563221  43.6896191  43.6435559
 43.7534804  43.6710244  43.6563221  43.6896191  43.7598243  43.7047983
 43.6563221  43.6499915  43.75646655 43.7996637  43.7543263  43.704553
 43.6563221  43.6602019  43.7437422  43.7543263  43.7543263  43.699971
 43.6563221  43.6449033  43.773077   43.7691966  43.7492988  43.6247901
 43.6563221  43.6449033  43.773077   43.7543263  43.7492988  43.6247901
 43.6563221  43.7543263  43.7600778  43.773077   43.7543263  43.7492988
 43.64958515 43.7543263  43.6896191  43.7543263  43.773077   43.7739798
 43.7492988  43.729199   43.7105408  43.6655802  43.7001608  43.773077
 43.7440391  43.697936   43.6449033  43.6449033  43.6936399  43.773077
 43.7739798  43.6449033  43.6449033  43.6398746  43.6668555  43.6435559
 43.7853531  43.697936   43.6563221  43.6517776  43.773077   43.6449

In [34]:
lon = np.asarray(lon, dtype = float)

In [35]:
df['latitude'] = lat
df['longitude'] = lon

In [36]:
df

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.656322,-79.380916
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.754326,-79.449117
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.656322,-79.380916
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.643556,-79.565633
99,M4Y,Downtown Toronto,Church and Wellesley,43.662693,-79.386875
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.624790,-79.393492
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.643556,-79.565633


# Explore and cluster the neighborhoods in Toronto